# 전국자동차종합정보 신규등록정보 서비스  

데이터 설명 
1. 제공 : 한국교통안전공단 
2. 출처 : https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15059401

- 코드 작성 : 맥북 os

In [4]:
import requests
#from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import time
import pandas as pd

In [9]:

service_key = 'api'



BASE = 'http://apis.data.go.kr/B553881/newRegistlnfoService_01/getnewRegistlnfoService'

In [6]:
fuel_codes = {
    '5': '전기',
    '7': '하이브리드(휘발유+전기)'
}


# 4) 지역 코드 및 이름
region_codes = {
    '1': '서울', '2': '부산', '3': '대구', '4': '인천', '5': '광주', '6': '대전', '7': '울산',
    '8': '세종', '9': '경기', '10': '강원', '11': '충북', '12': '충남', '13': '전북',
    '14': '전남', '15': '경북', '16': '경남', '17': '제주'
}

In [7]:
results = []

In [10]:
for year in range(2019, 2025):            # 2019–2024
    for month in range(1, 13):            # 월 1–12
        for fuel_code, fuel_name in fuel_codes.items():
            for region_code, region_name in region_codes.items():
                params = {
                    'ServiceKey':    service_key,
                    'registYy':      f"{year:04d}",
                    'registMt':      f"{month:02d}",
                    # 'vhctyAsortCode': '1', 
                    'registGrcCode': region_code,
                    'useFuelCode':   fuel_code
                }

                r = requests.get(BASE, params=params, timeout=5)
                root = ET.fromstring(r.content)
                cnt_str = root.findtext('.//body/dtaCo', default='0') or '0'
                try:
                    cnt = int(cnt_str)
                except ValueError:
                    cnt = 0

                results.append({
                    '연도':        year,
                    '월':         month,
                    '지역':       region_name,
                    '연료':       fuel_name,
                    '신규등록': cnt
                })
                time.sleep(0.05)

In [12]:
# 최종 DataFrame
df = pd.DataFrame(results)

In [54]:
df.head()

,연도,월,지역,연료,신규등록 수
0,2019,1,서울,전기,26
1,2019,1,부산,전기,2
2,2019,1,대구,전기,295
3,2019,1,인천,전기,6
4,2019,1,광주,전기,2


In [44]:

#df.to_excel('car_new_admin_0710.xlsx', index=False)


# DataBase에 저장하기

In [15]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pymysql 

In [53]:

connection = pymysql.connect(
    host = 'localhost',
    user = 'mountainbyeol',
    password = 'byeol617',
    database = 'PJ1_db',
    charset='utf8mb4',
    autocommit=False
    )


try:
    with connection.cursor() as cursor:
        #테이블 생성 
        #cursor.execute()
        
        sql = "INSERT INTO car_new1 (`year`, `month`, `region`, `fuel`, `new_admin`) VALUES( %s,%s,%s,%s,%s)"
        for data in results:
            cursor.execute(sql,
                          ( data['연도'],
                           data['월'],
                           data['지역'],
                           data['연료'],
                           data['신규등록 수']
                          )                           
                          )
    connection.commit()


finally:
    connection.close()
                           